<a href="https://colab.research.google.com/github/FelipeNemo/email-validation-guide/blob/main/emailvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sistema de Validação de Email para prevenção de fraude**

**Autores:** Felipe Augusto Batista Mendes dos Santos



Este sistema atende à necessidade de validação de e-mails para aumentar a eficiência e a segurança das transações online, utilizando um pipeline de processamento. Ele visa usar o status de classificação de validade para a tomada de decisões.

* **Input:** Dados estruturados
* **Output:** Status de classificação de e-mail

# Verificação de Email
Verificação de um endereço de email existente e ativo, ou seja, válido. Um email válido pode receber mensagens de outros remetentes. São responsáveis por executar uma etapa do **PROCESSO DE VERIFICAÇÃO**, como por exemplo:
* Verificações de sintaxe
* Domínio
* Registro MX
* Catch-all
* Ping de endereço de e-mail

# Validação de email
É o resultado final do processo de verificação de email. Às vezes, a validação também implica na verificação de qualquer informação adicional que vem com o e-mail do lead ou na verificação da origem desse lead.

* “Não verificado” ⭕ - antes do processo de verificação.

* “Válido” 🟢 - significa que um e-mail está correto e existe.

* “Inválido” 🟡 -  significa que esse endereço de e-mail pode receber mensagens, mas devido a uma configuração em todo o domínio, não é possível determinar a sua validade real, assim ele é marcado como Não verificável. Esses e-mails também são conhecidos como e-mails “pegam tudo” ou “aceitam tudo”.

* “Não verificável” 🔴 – significa que um endereço de e-mail não existe ou não está mais ativo.

*  “Não é possível encontrar o e-mail” ⚪ - clientes potenciais com informações pessoais, como fotos ou experiência profissional etc., no entanto, não há endereço de e-mail.


# **Não verificado ⭕ :**
– 1. Ainda não entrou em nenhum processo de validação

# **Não verificável 🔴 :**
– 1. O e-mail falha na verificação de sintaxe (etapa 2).

– 2. O domínio não possui um registro A válido (etapa 3).

– 3. O domínio não possui registros MX válidos (etapa 4).

– 4. O domínio não responde à verificação SMTP (etapa 5).

# **Não é possível encontrar o e-mail ⚪ :**
– 1. O e-mail não contém exatamente um símbolo "@" ou outras informações essenciais estão ausentes, impedindo a verificação de sintaxe (etapa2).         
# **Inválido 🟡 :**
– 1. O e-mail pode receber mensagens, mas devido a uma configuração em todo o domínio (catch-all), não é possível determinar sua validade real (etapa 4).

# **Válido 🟢 :**
– 1. Sintaxe correta (etapa 2).

– 2. Domínio existente possui registro A válido (etapa 3).

– 3. Domínio possui registros MX válidos (etapa 4).

– 4. Domínio não está configurado como catch-all (etapa 4).

– 5. Domínio responde à verificação SMTP (etapa 5).

# Para que verificar emails?

1.    Manter listas de e-mail atualizadas
2.    Reduz hard bounces
3.    Melhora a capacidade de entrega de e-mail
4.    Mantém a reputação do remetente elevada
4.    Prevenção de ataque de fraude










# Como funciona um verificador de e-mail:
1.   Verificação de sintaxe
2.   Verificação de jargão
3.   Verificação de existência de domínio
4.   Verificação de registro MX
5.   Verificação de domínio catch-all
6.   Autenticação SMTP (ping de endereço de e-mail)
7.   Verificação de freemail




# **Install:**

In [3]:
!pip install dnspython requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 1.9 MB/s eta 0:00:00


# **Importações:**

In [4]:
import csv
from functools import reduce
from itertools import *
import dns.resolver # modulo de dnspython
import requests
import smtplib

# **Leitura .csv**

In [5]:
def mostra_dados(titulo, nome_arq, f, *args):
    """
    Função auxiliar para mostrar o resultado da execução de uma função do paradigma funcional.
    A função abre um arquivo CSV e passa para função do paradigma funcional um objeto DictReader e
    depois mostra o resultado na tela.
    """
    print("|___   ", titulo, "  ___|")
    with open(nome_arq, "r", encoding='utf-8-sig') as arq:
        dados = csv.DictReader(arq, delimiter=',')
        fdados = f(dados, *args)
        try:
            iterator = iter(fdados)
        except TypeError:
            print(fdados)
        else:
            for linha in fdados:
                print(linha)

def retorna_dados(nome_arq):
    """
    Função auxiliar que retorna os dados em formato de um objeto DictReader após a leitura de um arquivo!
    """
    with open(nome_arq, "r", encoding='utf-8-sig') as arq:
        dados = csv.DictReader(arq, delimiter=',')
        for linha in dados:
            for chave, valor in linha.items():
                linha[chave] = valor.strip()  # reassign
            yield linha

def salvar_dados(nome_arq, new_file, f):
    """
    Função auxiliar que chama uma função do paradigma funcional e depois salva o resultado em um arquivo CSV!
    A função abre um arquivo CSV e passa para função do paradigma funcional um objeto DictReader e
    depois salva a saída em um arquivo CSV
    """
    with open(nome_arq, "r", encoding='utf-8-sig') as arq:
        dados=csv.DictReader(arq, delimiter=',')
        fdados = list(f(dados))
        fieldnames = fdados[0].keys()

    with open(new_file, "w", encoding='utf-8-sig') as arq:
        try:
            iterator=iter(fdados)
        except TypeError:
            return false
        else:
            writer = csv.DictWriter(arq, fieldnames=fieldnames)
            writer.writeheader()
            for i in fdados:
                writer.writerow(i)

# **1. Verificação de sintaxe:**
**Ferramentas:**  

*   Python

## **Correspondência de padrão de email:**

> < local-part@domain >

1. **local-part** -  Pode ter até 64 octetos de comprimento, disposição de pontos e caracteres especiais.

2. **@** - Deve conter apenas 1 elemento "@"em todo o e-mail

3. **domain** -  Pode ter no máximo 255 octetos e deve conter apenas 1 "."



### **1.1 Local-part:**
**Ferramentas:**

*   dnspython
 **Disposição de bytes:**
*  64 octetos de comprimento

**Disposição geral de pontos:**

 Não é permitido que seja o primeiro ou o último caractere, e que também apareça consecutivamente

* **Caracteres especiais:**

> "() , : ; < > @ [ \ ]

  -  São permitidos com restrições(somente dentro de uma string entre aspas e nessa string entre aspas, qualquer barra invertida ou aspas duplas deve ser precedida uma vez por uma barra invertida)
   
   Exemplo:
   
   john.smith(comment)@example.come e (comment)john.smith@example.com são ambos equivalentes a john.smith@example.com

   

In [6]:
#Constantes:
CARACTERES_ESPECIAIS = { "(", ")", ",", ":", ";", "<", ">", "@", "[", "]" }

In [7]:

def local_sintax(local):
    print(f"Verificando parte local: {local}")
    # Verifica o comprimento da parte local
    if len(local.encode('utf-8')) > 64:
        print("Falhou: comprimento maior que 64 bytes")
        return False

    # Verifica a disposição de pontos na parte local
    if local.startswith('.') or local.endswith('.'):
        print("Falhou: começa ou termina com '.'")
        return False
    if '..' in local:
        print("Falhou: contém '..'")
        return False

    # Verifica a presença de caracteres especiais na parte local
    if any(char in CARACTERES_ESPECIAIS for char in local):
        if not (local.startswith('"') and local.endswith('"')):
            print("Falhou: contém caracteres especiais fora de aspas")
            return False
        for i, char in enumerate(local):
            if char in CARACTERES_ESPECIAIS:
                if i == 0 or i == len(local) - 1 or local[i - 1] != '\\':
                    print("Falhou: caracteres especiais não escapados")
                    return False
    return True

### **1.2 Domain**
**Ferramentas:**  

*   Python

 **Disposição de bytes:**
* 255 octetos de comprimento

**Disposição geral de pontos:**
  
* Deve conter apenas 1 "." :

In [8]:
def domain_sintax(domain):
    print(f"Verificando parte do domínio: {domain}")
    # Verifica o comprimento da parte do domínio
    if len(domain.encode('utf-8')) > 255:
        print("Falhou: comprimento maior que 255 bytes")
        return False

    # Verifica se o domínio contém pelo menos um "."
    if '.' not in domain:
        print("Falhou: não contém '.'")
        return False

    return True

### **Conclusão:**

In [9]:
def verificador_sintax(email, local_sintax, domain_sintax):
    # Verifica se contém exatamente um "@"
    if email.count('@') != 1:
        print("Falhou: não contém exatamente um '@'")
        return False

    local, domain = email.split('@')

    # Verificação da parte local
    if not local_sintax(local):
        print("Falhou: verificação da parte local")
        return False

    # Verificação da parte do domínio
    if not domain_sintax(domain):
        print("Falhou: verificação da parte do domínio")
        return False

    return True



In [10]:
# Teste do email
email = "john.smith@example.com"
resultado = verificador_sintax(email, local_sintax, domain_sintax)
print(f"O email '{email}' é válido? {resultado}")

Verificando parte local: john.smith
Verificando parte do domínio: example.com
O email 'john.smith@example.com' é válido? True


# **2. Verificação de domínio:**
**Ferramentas:**

*   dnspython


In [11]:
# Função para verificar se o domínio existe
def check_domain_exists(domain):
    try:
        # Verifica registros DNS
        dns.resolver.resolve(domain, 'A')
        return True
    except (dns.resolver.NXDOMAIN, dns.resolver.NoAnswer, dns.resolver.Timeout):
        return False

# Função para verificar se o domínio está ativo
def check_domain_active(domain):
    try:
        # Faz uma requisição HTTP ao domínio
        response = requests.get(f"http://{domain}", timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

# Exemplo de uso
domain = "gmail.com"
exists = check_domain_exists(domain)
active = check_domain_active(domain)

print(f"O domínio '{domain}' existe? {exists}")
print(f"O domínio '{domain}' está ativo? {active}")

O domínio 'gmail.com' existe? True
O domínio 'gmail.com' está ativo? True


# **3. Verificação do registro MX:**
**Ferramentas:**

*   dnspython

O registro MX especifica o servidor de e-mail que é responsável por aceitar emails em nome de um domínio específico.Ele funciona verificando se a entrada de troca de correio listada no endereço de e-mail é um nome de domínio real, registrado e hospedado, que pode ser acessado online.

In [19]:
def get_mx_records(domain):
    try:
        answers = dns.resolver.resolve(domain, 'MX')
        mx_records = sorted([(rdata.preference, str(rdata.exchange)) for rdata in answers], key=lambda x: x[0])
        return [record[1] for record in mx_records]
    except dns.resolver.NoAnswer:
        print(f"Não há registros MX para o domínio: {domain}")
        return None
    except dns.resolver.NXDOMAIN:
        print(f"O domínio {domain} não existe.")
        return None
    except dns.resolver.Timeout:
        print(f"Timeout ao consultar o domínio: {domain}")
        return None
    except dns.resolver.NoNameservers:
        print(f"Os servidores de nomes não puderam ser encontrados para o domínio: {domain}")
        return None
    except dns.resolver.YXDOMAIN:
        print(f"Nome do domínio muito longo para: {domain}")
        return None
    except dns.resolver.NoRootSOA:
        print(f"Domínio {domain} não possui uma SOA válida.")
        return None
    except dns.exception.DNSException as e:
        print(f"Erro ao consultar os registros MX do domínio {domain}: {e}")
        return None

In [20]:
# Exemplo de uso
domain = "gmail.com"
mx_records = get_mx_records(domain)

if mx_records:
    print(f"Registros MX para {domain}:")
    for mx in mx_records:
        print(f"- {mx}")
else:
    print(f"Não foi possível encontrar registros MX para o domínio: {domain}")

Registros MX para gmail.com:
- gmail-smtp-in.l.google.com.
- alt1.gmail-smtp-in.l.google.com.
- alt2.gmail-smtp-in.l.google.com.
- alt3.gmail-smtp-in.l.google.com.
- alt4.gmail-smtp-in.l.google.com.


# **4.Verificação catch-all:**
**Ferramentas:**

*   dnspython
*   smtplib

**O que é um domínio "catch-all":**

 está configurado para aceitar emails enviados para qualquer endereço no domínio, mesmo que o endereço específico não exista.

In [14]:
def check_catch_all(domain):
    mx_records = get_mx_records(domain)
    if not mx_records:
        print(f"Não foi possível encontrar registros MX para o domínio: {domain}")
        return False

    test_email = f"catchall-test-{domain}@{domain}"
    sender_email = "test@example.com"

    for mx in mx_records:
        try:
            server = smtplib.SMTP(mx)
            server.set_debuglevel(0)  # Defina para 1 se quiser ver a comunicação SMTP
            server.ehlo_or_helo_if_needed()
            server.mail(sender_email)
            code, message = server.rcpt(test_email)
            server.quit()

            if code == 250:
                print(f"O domínio {domain} possui um catch-all configurado.")
                return True
        except smtplib.SMTPServerDisconnected:
            pass
        except smtplib.SMTPConnectError:
            pass
        except smtplib.SMTPRecipientsRefused:
            pass
        except Exception as e:
            print(f"Erro ao verificar o domínio {domain}: {e}")
            return False

    print(f"O domínio {domain} não possui um catch-all configurado.")
    return False

In [15]:
# Exemplo de uso
domain = "gmail.com"
catch_all = check_catch_all(domain)
print(f"Catch-all configurado para {domain}? {catch_all}")

O domínio gmail.com não possui um catch-all configurado.
Catch-all configurado para gmail.com? False


# **5. Ping do endereço de e-mail:**
**Ferramentas:**

*   dnspython
*   smtplib

In [16]:
def ping_email_address(email):
    domain = email.split('@')[1]
    mx_records = get_mx_records(domain)
    if not mx_records:
        print(f"Não foi possível encontrar registros MX para o domínio: {domain}")
        return False

    test_email = email
    sender_email = "test@example.com"  # Pode ser qualquer endereço válido

    for mx in mx_records:
        try:
            # Conectar ao servidor SMTP
            server = smtplib.SMTP(mx, timeout=10)
            server.set_debuglevel(0)  # Defina para 1 se quiser ver a comunicação SMTP
            server.helo()
            server.mail(sender_email)
            code, message = server.rcpt(test_email)
            server.quit()

            if code == 250:
                print(f"O e-mail {email} parece ser válido.")
                return True
            else:
                print(f"O e-mail {email} não parece ser válido. Código de resposta: {code}")
                return False
        except smtplib.SMTPConnectError:
            print(f"Erro ao conectar com o servidor de e-mail {mx}.")
        except Exception as e:
            print(f"Erro ao verificar o e-mail {email}: {e}")

    print(f"Não foi possível verificar o e-mail {email}.")
    return False


In [17]:
# Exemplo de uso
email = "felipedata20@gmail.com"
is_valid = ping_email_address(email)
print(f"Endereço de e-mail {email} é válido? {is_valid}")

O e-mail felipedata20@gmail.com parece ser válido.
Endereço de e-mail felipedata20@gmail.com é válido? True


# **Futuras melhorias:**


# **Documentações:**

*   itertools: https://docs.python.org/3/library/itertools.html#itertools.count
*   dsnpython: https://dnspython.readthedocs.io/en/latest/



## Referências:
* Principal Artigo de referência:

 https://snov.io/blog/br/verificacao-de-email-explicada/
*   Plataforma de validação de email que inspirou o projeto:

 https://snov.io/br/verificador-de-emails
*  Entendendo DNS:

 https://support.google.com/a/answer/48090?hl=pt-BR#:~:text=voltar%20ao%20in%C3%ADcio-,Registro%20MX,de%20e%2Dmail%20do%20Google.
*   Status de email:

 https://snov.io/knowledgebase/email-statuses/

*   Definições de leads:

 https://fleeg.com/blog/inbound-marketing/leads/?origin=fleeg_snovio_link_backlink&utm_source=Snovio&utm_medium=link&utm_campaign=backlink
*   Verificação sintaxe:

 https://en.wikipedia.org/wiki/Email_address#Local-part

 https://en.wikipedia.org/wiki/Octet_(computing)

 https://www.youtube.com/watch?v=WkNBhYMUXwI
*   Envio de email SMTP:

 https://www.youtube.com/watch?v=umvzsQLZYD4
*   Verificação TLD/MX record/Sintaxe:

 https://www.youtube.com/watch?v=Bj61jJo2Mvs
*   Item da lista
*   Item da lista



